### Configs and Libs

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_1_1'
CRYPTO_TARGET = 'NONE'

### Creating a New Directory

The following bash block will create a new directory and copy the `simpleserial-base` code into it:

In [2]:
'''%%bash
cd ../../../../hardware/victims/firmware/
mkdir -p ops_with_key && cp -r simpleserial-base/* $_
cd ops_with_key'''

'%%bash\ncd ../../../hardware/victims/firmware/\nmkdir -p ops_with_key && cp -r simpleserial-base/* $_\ncd ops_with_key'

### Connecting to Hardware
Quand j'éxecute cette ligne le scope est defini

In [3]:
%run "../../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [4]:
print(target)
print(scope)

SimpleSerial Settings =
	output_len             = 16
	baud                   = 38400
	simpleserial_last_read = 
	simpleserial_last_sent = 
	
ChipWhisperer Nano Device
fw_version = 
    major = 0
    minor = 65
    debug = 0
io = 
    tio1         = None
    tio2         = None
    tio3         = None
    tio4         = high_z
    pdid         = True
    pdic         = False
    nrst         = True
    clkout       = 7500000.0
    cdc_settings = bytearray(b'\x01\x00\x00\x00')
adc = 
    clk_src  = int
    clk_freq = 7500000.0
    samples  = 5000
glitch = 
    repeat     = 0
    ext_offset = 0



### Building Firmware

Rerun this every time you make changes to your code:

In [5]:
%%bash -s "$PLATFORM" "$SS_VER" "$CRYPTO_TARGET"
cd ../../../../hardware/victims/firmware/rec-s-from-leaked-data
make PLATFORM=$1 SS_VER=$2 CRYPTO_TARGET=$3 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
arm-none-eabi-gcc.exe (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy -I.././crypto/ .././hal/stm32f0/stm32f0_startup.S -o

### Uploading Firmware


In [6]:
cw.program_target(scope, prog, "../../../../hardware/victims/firmware/rec-s-from-leaked-data/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4443 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4443 bytes


## Trace Capture

In [7]:
#validado
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
textin_array = []
textout_array = []

N = 1000
for i in trange(N, desc='Capturing data'):
    key, text = ktp.next() 
    trace = cw.capture_trace(scope, target, text)

    if not trace:
        continue

    textout_array.append(trace.textout)
    textin_array.append(trace.textin)

Capturing data:   0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
print("textin_array[i][0]")
for i in range (3) : 
       print(hex(textin_array[i][0]))


textin_array[i][0]
0x41
0x56
0x6


In [14]:
print("textout_array[i][0]")
for i in range (3) : 
       print(hex(textout_array[i][0]))

textout_array[i][0]
0xff
0x14
0xc4


In [11]:
assert(len(textout_array) == N)
assert(len(textin_array) == N)


In [12]:
scope.dis()
target.dis()